### imports & utils

In [35]:
import pandas as pd
import numpy as np
from pathlib import Path
from tabular.dl_logs import  CALMIP_START_INDEX

import json
import itertools
import copy

%reload_ext autoreload
%autoreload 2

SCRIPT_PATH = Path('./scripts/calmip')
LOC_PATH = Path('./scripts/local')
LOG_FILE = Path('./logs/calmip_logs.csv')

In [36]:
def create_model_identifier(row):
    return f"{row['model']}_{row['features']}_{row['seq_len']}-{row['pred_len']}_el{row['e_layers']}_dm{row['d_model']}_nh{row['n_heads']}_dl{row['d_layers']}_ff{row['d_ff']}_do{row['dropout']}"


### .json config for pruning

In [37]:
ths = [16,32,64]
features = ['MS','S']

common_hps = {
    'dropout':[0.1,0.2,0.5], 
    'wd':[0.05,0.1],
    'learning_rate' : [0.0005]
}

model_specific_hps = {
    'Transformer' : {'e_layers':[1], 'd_model':[512], 'd_layers' : [4]},
    # 'Transformer2D' : {'e_layers':[3,4,6], 'd_model':[512], 'd_layers' : [1,2,3,4]},
    # 'PatchTST' : {'e_layers':[3,4,6], 'd_model':[256], 'd_layers' : [1,2]},
    # 'Nonstationary_Transformer' : {'e_layers':[1,2,3], 'd_model':[512], 'd_layers' : [1,2,3]},
    # 'iTransformer' : {'e_layers':[1,2,4], 'd_model':[512], 'd_layers' : [1,2]},
    # 'TimesNet' : {'e_layers':[2], 'd_model':[512], 'd_layers' : [1]},
}

In [38]:
c_keys = common_hps.keys()
c_values = common_hps.values()
c_combinations = itertools.product(*c_values)
c_configs = [dict(zip(c_keys, combination)) for combination in c_combinations]
len(c_configs)

6

In [39]:
for th in ths:
    for ft in features :
        filename = f"experiments_list_th{th}_{ft}.json"
        configs = []
        ms_configs = []
        for model, model_hp in model_specific_hps.items():
            ms_keys = model_hp.keys()
            ms_values = model_hp.values()
            ms_combinations = itertools.product(*ms_values)
            tmp_dict = [dict(zip(ms_keys, combination)) for combination in ms_combinations]
            for ms_c in tmp_dict : 
                ms_c['model'] = model
                ms_c['d_ff'] = int(ms_c['d_model'] /2)
                ms_configs.append(ms_c)
        for c_config in c_configs :
            for ms_config in ms_configs :
                conc_config = copy.deepcopy(ms_config)    
                conc_config.update(c_config)
                configs.append(conc_config) 
        with open(filename, 'w') as f:
            # indent=4 makes the output nicely formatted
            json.dump(configs, f, indent=4)        
print(len(ms_configs), len(c_config))
print(len(configs))

1 3
6


In [40]:
configs

[{'e_layers': 1,
  'd_model': 512,
  'd_layers': 4,
  'model': 'Transformer',
  'd_ff': 256,
  'dropout': 0.1,
  'wd': 0.05,
  'learning_rate': 0.0005},
 {'e_layers': 1,
  'd_model': 512,
  'd_layers': 4,
  'model': 'Transformer',
  'd_ff': 256,
  'dropout': 0.1,
  'wd': 0.1,
  'learning_rate': 0.0005},
 {'e_layers': 1,
  'd_model': 512,
  'd_layers': 4,
  'model': 'Transformer',
  'd_ff': 256,
  'dropout': 0.2,
  'wd': 0.05,
  'learning_rate': 0.0005},
 {'e_layers': 1,
  'd_model': 512,
  'd_layers': 4,
  'model': 'Transformer',
  'd_ff': 256,
  'dropout': 0.2,
  'wd': 0.1,
  'learning_rate': 0.0005},
 {'e_layers': 1,
  'd_model': 512,
  'd_layers': 4,
  'model': 'Transformer',
  'd_ff': 256,
  'dropout': 0.5,
  'wd': 0.05,
  'learning_rate': 0.0005},
 {'e_layers': 1,
  'd_model': 512,
  'd_layers': 4,
  'model': 'Transformer',
  'd_ff': 256,
  'dropout': 0.5,
  'wd': 0.1,
  'learning_rate': 0.0005}]

### args creation from logs

In [3]:
common_args = ["--task_name=long_term_forecast",
               "--is_training= 1",
               "--root_path = /tmpdir/arnaud/dataset/USC/",
               "--checkpoints = /tmpdir/arnaud/checkpoints/",
               "--results_path = /tmpdir/arnaud/results/",
               "--data_path = na",
               "--data = USC",
               "--des = Exp",
               "--batch_size = 32",
               "--train_epochs = 48",
               "--patience= 16",
               "--embed=fixed",
               "--consider_only_offense",
               "--inverse",
               "--itr= 3"]


In [4]:
ldf = pd.read_csv(LOG_FILE, index_col=0)
ldf = ldf[(ldf.index >= CALMIP_START_INDEX[2]) & (ldf.index < CALMIP_START_INDEX[4])]
ldf

,task_name,is_training,model_id,model,data,root_path,data_path,features,target,freq,...,nb_params,nb_tr_params,nb_nontr_params,mae,mse,rmse,mape,mspe,fde,results_path
471,long_term_forecast,1,USC_16_16,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,0.614937,0.878844,0.937467,inf,inf,1.597670,/tmpdir/arnaud/results
472,long_term_forecast,1,USC_24_24,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,0.913752,2.036071,1.426909,inf,inf,2.472808,/tmpdir/arnaud/results
473,long_term_forecast,1,USC_48_48,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,1.647258,7.078037,2.660458,inf,inf,4.432114,/tmpdir/arnaud/results
474,long_term_forecast,1,USC_32_32,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,1.195554,3.562750,1.887525,inf,inf,3.252716,/tmpdir/arnaud/results
475,long_term_forecast,1,USC_16_16,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,0.607831,0.871429,0.933504,inf,inf,1.594972,/tmpdir/arnaud/results
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,long_term_forecast,1,'USC_96_96',Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,5283585,2684673,2598912,2.655750,16.311287,4.038723,inf,inf,6.184798,/tmpdir/arnaud/results/
1600,long_term_forecast,1,'USC_96_96',Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,4492801,1893889,2598912,2.778857,15.790590,3.973737,inf,inf,5.891378,/tmpdir/arnaud/results/
1601,long_term_forecast,1,'USC_96_96',Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,5283585,2684673,2598912,2.690717,15.733384,3.966533,inf,inf,5.999330,/tmpdir/arnaud/results/
1602,long_term_forecast,1,'USC_96_96',Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,5810689,3211777,2598912,2.643830,16.859442,4.106025,inf,inf,6.390666,/tmpdir/arnaud/results/


In [5]:
ldf['exp_id'] = ldf.apply(lambda row : create_model_identifier(row), axis =1)
edf = ldf[(ldf["seq_len"]==16) & (ldf["features"]=="S") ][['exp_id',"fde","features",'fit_time','epoch']].groupby('exp_id').agg({'fde':'count','features':'count','fit_time':'sum', 'epoch' : 'mean'})
edf['fit_time'] = edf['fit_time'] / 3600
edf

,fde,features,fit_time,epoch
exp_id,,,,
DLinear_S_16-16_el2_dm512_nh8_dl1_ff2048_do0.1,3,3,0.734414,38.000000
LSTM_S_16-16_el1_dm256_nh8_dl1_ff512_do0.1,3,3,0.808700,46.000000
LSTM_S_16-16_el1_dm64_nh8_dl1_ff64_do0.1,3,3,0.796625,45.333333
LSTM_S_16-16_el2_dm256_nh8_dl1_ff512_do0.1,3,3,0.804517,36.666667
LSTM_S_16-16_el2_dm64_nh8_dl1_ff64_do0.1,3,3,0.848978,46.000000
LSTM_S_16-16_el2_dm64_nh8_dl2_ff64_do0.1,3,3,0.897356,46.666667
Nonstationary_Transformer_S_16-16_el2_dm128_nh8_dl1_ff128_do0.1,3,3,4.479383,34.333333
Nonstationary_Transformer_S_16-16_el2_dm256_nh8_dl1_ff256_do0.1,3,3,4.488947,37.333333
Nonstationary_Transformer_S_16-16_el2_dm64_nh4_dl1_ff64_do0.1,3,3,4.476750,39.666667


In [6]:
time_limits = {
    'LSTM' : '02:00:00',
    'Transformer' :'08:00:00',
    'iTransformer' :'02:00:00',
    'PatchTST' :'06:00:00',
    'Nonstationary_Transformer' :'06:00:00',
    'TimesNet' :'06:00:00',
    'DLinear' :'02:00:00'    
}

In [7]:
tmp_df = ldf[(ldf["seq_len"]==16) & (ldf["features"]=="S") ].copy()

configs = []
config_df = tmp_df[['model','dropout','learning_rate','optimizer', "wd","e_layers",'d_model','n_heads', 'd_layers', 'd_ff']].copy()

In [8]:
configs = [v for v in config_df.drop_duplicates().T.to_dict().values()]


In [9]:
configs_list = [[f"--{k}={v}" for k,v in config.items()]for config in configs]
print(len(configs_list))
configs_list[0]

27


['--model=LSTM',
 '--dropout=0.1',
 '--learning_rate=0.001',
 '--optimizer=adamw',
 '--wd=0.05',
 '--e_layers=2',
 '--d_model=256',
 '--n_heads=8',
 '--d_layers=1',
 '--d_ff=512']

In [10]:
exp_count = ldf.groupby('exp_id').count()['features']
exp_count

exp_id
DLinear_MS_16-16_el2_dm512_nh8_dl1_ff2048_do0.1       3
DLinear_MS_24-24_el2_dm512_nh8_dl1_ff2048_do0.1       3
DLinear_MS_32-32_el2_dm512_nh8_dl1_ff2048_do0.1       3
DLinear_MS_48-48_el2_dm512_nh8_dl1_ff2048_do0.1       3
DLinear_MS_64-64_el2_dm512_nh8_dl1_ff2048_do0.1       3
                                                     ..
iTransformer_S_96-96_el2_dm512_nh4_dl1_ff512_do0.1    3
iTransformer_S_96-96_el3_dm256_nh4_dl1_ff256_do0.1    3
iTransformer_S_96-96_el3_dm512_nh4_dl1_ff512_do0.1    3
iTransformer_S_96-96_el4_dm256_nh8_dl1_ff256_do0.1    3
iTransformer_S_96-96_el4_dm512_nh8_dl1_ff512_do0.1    3
Name: features, Length: 306, dtype: int64

In [11]:
features_set = {
   'S' : ["--features=S", "--enc_in=1", "--dec_in=1", "--c_out=1"],
   'MS' : ["--features=MS", "--enc_in=61", "--dec_in=61", "--c_out=61"]
}
time_horizons = [16,32,64]


### Target jobs

In [12]:
with open(SCRIPT_PATH / 'jobs_configs.txt', "w") as f:
    i = 0
    for th in time_horizons :
        th_args = [f"--model_id='USC_{th}_{th}'", f"--seq_len={th}", f"--pred_len={th}", f"--label_len={th//2}"]
        for feature, feature_args in features_set.items():
            for config_list, config_dict in zip(configs_list, configs) :
                args = th_args + feature_args + config_list + common_args
                identifier = f"{config_dict['model']}_{feature}_{th}-{th}_el{config_dict['e_layers']}_dm{config_dict['d_model']}_nh{config_dict['n_heads']}_dl{config_dict['d_layers']}_ff{config_dict['d_ff']}"
                str_args = f"{i}, {identifier}, {time_limits[config_dict['model']]}," + ",".join(args)
                str_args = str_args.replace(' ','')
                # Add IF condition here if needed
                print(str_args)
                i+=1
                f.write(str_args)
                f.write("\n")
        

0,LSTM_S_16-16_el2_dm256_nh8_dl1_ff512,02:00:00,--model_id='USC_16_16',--seq_len=16,--pred_len=16,--label_len=8,--features=S,--enc_in=1,--dec_in=1,--c_out=1,--model=LSTM,--dropout=0.1,--learning_rate=0.001,--optimizer=adamw,--wd=0.05,--e_layers=2,--d_model=256,--n_heads=8,--d_layers=1,--d_ff=512,--task_name=long_term_forecast,--is_training=1,--root_path=/tmpdir/arnaud/dataset/USC/,--checkpoints=/tmpdir/arnaud/checkpoints/,--results_path=/tmpdir/arnaud/results/,--data_path=na,--data=USC,--des=Exp,--batch_size=32,--train_epochs=48,--patience=16,--embed=fixed,--consider_only_offense,--inverse,--itr=3
1,LSTM_S_16-16_el1_dm256_nh8_dl1_ff512,02:00:00,--model_id='USC_16_16',--seq_len=16,--pred_len=16,--label_len=8,--features=S,--enc_in=1,--dec_in=1,--c_out=1,--model=LSTM,--dropout=0.1,--learning_rate=0.001,--optimizer=adamw,--wd=0.05,--e_layers=1,--d_model=256,--n_heads=8,--d_layers=1,--d_ff=512,--task_name=long_term_forecast,--is_training=1,--root_path=/tmpdir/arnaud/dataset/USC/,--checkpoin

### Local jobs

In [17]:
last_completed_index = 34

In [12]:
loc_common_args = ["--task_name=long_term_forecast",
               "--is_training= 1",
               "--root_path= ./dataset/USC/",
               "--checkpoints= ./checkpoints/",
               "--results_path= ./results/",
               "--data_path = na",
               "--data = USC",
               "--des = Exp",
               "--batch_size = 128",
               "--train_epochs = 12",
               "--patience= 4",
               "--embed=fixed",
               "--consider_only_offense",
               "--inverse",
               "--itr= 1"]

In [13]:
ths = [32]
fs = ['MS']
models_hps = {
    'Transformer' : {'e_layers':[1,2,3,4,6], 'd_model':[64,256,512], 'd_layers' : [1,2,3,4], 'learning_rate' : [0.0005], 'dropout':[0.1,0.2,0.5], 'wd':[0.05,0.1,0.01] },
    'Transformer2D' : {'e_layers':[1,2,4,6], 'd_model':[64,256,512], 'd_layers' : [1,2,3,4],'learning_rate' : [0.0005], 'dropout':[0.1,0.2,0.5], 'wd':[0.05,0.1,0.01] },
    'PatchTST' : {'e_layers':[1,2,4], 'd_model':[64,256], 'd_layers' : [1,2],'learning_rate' : [0.0005], 'dropout':[0.1,0.2,0.5], 'wd':[0.05,0.1,0.01]},
    'Nonstationary_Transformer' : {'e_layers':[1,2,4], 'd_model':[64,256,512], 'd_layers' : [1,2],'learning_rate' : [0.0001], 'dropout':[0.1,0.2,0.5], 'wd':[0.05,0.1,0.01] },
    'iTransformer' : {'e_layers':[1,2,4], 'd_model':[64,256,512], 'd_layers' : [1,2], 'learning_rate' : [0.0005], 'dropout':[0.1,0.2,0.5], 'wd':[0.05,0.1,0.01] },
    'TimesNet' : {'e_layers':[2], 'd_model':[512], 'd_layers' : [1], 'learning_rate' : [0.001], 'dropout':[0.1,0.2,0.5], 'wd':[0.05,0.1,0.01] },
}
ls = ['MSE']

In [14]:
with open(LOC_PATH / 'local_jobs_configs.txt', "w") as f:
    i = 0
    for th in ths :
        th_args = [f"--model_id=USC_{th}_{th}", f"--seq_len={th}", f"--pred_len={th}", f"--label_len={th//2}"]
        for ft in fs:
            feature_args = features_set[ft]
            for model, model_hp in models_hps.items() :
                for el in model_hp['e_layers'] :
                    for dm in model_hp['d_model'] :
                        for dl in model_hp['d_layers'] :
                            if dl > el :
                                pass
                            else :
                                for lr in model_hp['learning_rate'] :
                                    for loss in ls :
                                        for lrs in [True, False]:
                                            config_args = [f"--model={model}",f"--e_layers={el}", f"--d_model={dm}", "--n_heads=8" ,f"--d_layers={dl}", f"--d_ff={dm//2}", f"--learning_rate={lr}", f'--loss={loss}']
                                            args = th_args + feature_args + config_args + loc_common_args 
                                            if lrs :
                                                args += ["--lr_scheduler"]
                                            identifier = f"{model}_{ft}_{th}-{th}_el{el}_dm{dm}_nh8_dl{dl}_ff{dm//2}"
                                            str_args = f"{i}, {identifier}, n.a.," + ",".join(args)
                                            str_args = str_args.replace(' ','')
                                            # Add IF condition here if needed
                                            i+=1

                                            print(str_args)
                                            f.write(str_args)
                                            f.write("\n")

0,Transformer_MS_32-32_el1_dm64_nh8_dl1_ff32,n.a.,--model_id=USC_32_32,--seq_len=32,--pred_len=32,--label_len=16,--features=MS,--enc_in=61,--dec_in=61,--c_out=61,--model=Transformer,--e_layers=1,--d_model=64,--n_heads=8,--d_layers=1,--d_ff=32,--learning_rate=0.0005,--loss=MSE,--task_name=long_term_forecast,--is_training=1,--root_path=./dataset/USC/,--checkpoints=./checkpoints/,--results_path=./results/,--data_path=na,--data=USC,--des=Exp,--batch_size=128,--train_epochs=12,--patience=4,--embed=fixed,--consider_only_offense,--inverse,--itr=1,--lr_scheduler
1,Transformer_MS_32-32_el1_dm64_nh8_dl1_ff32,n.a.,--model_id=USC_32_32,--seq_len=32,--pred_len=32,--label_len=16,--features=MS,--enc_in=61,--dec_in=61,--c_out=61,--model=Transformer,--e_layers=1,--d_model=64,--n_heads=8,--d_layers=1,--d_ff=32,--learning_rate=0.0005,--loss=MSE,--task_name=long_term_forecast,--is_training=1,--root_path=./dataset/USC/,--checkpoints=./checkpoints/,--results_path=./results/,--data_path=na,--data=USC,--des=E

### All jobs

In [33]:
features_set = {
   'S' : ["--features=S", "--enc_in=1", "--dec_in=1", "--c_out=1"],
   'MS' : ["--features=MS", "--enc_in=61", "--dec_in=61", "--c_out=61"]
}
time_horizons = [16,32,64]

with open(SCRIPT_PATH / 'all_jobs_configs.txt', "w") as f:
    i = 0
    for th in time_horizons :
        th_args = [f"--model_id='USC_{th}_{th}'", f"--seq_len={th}", f"--pred_len={th}", f"--label_len={th//4}"]
        for feature, feature_args in features_set.items():
            for config_list, config_dict in zip(configs_list, configs) :
                args = th_args + feature_args + config_list + common_args
                identifier = f"{config_dict['model']}_{feature}_{th}-{th}_el{config_dict['e_layers']}_dm{config_dict['d_model']}_nh{config_dict['n_heads']}_dl{config_dict['d_layers']}_ff{config_dict['d_ff']}"
                str_args = f"{i}, {identifier}, {time_limits[config_dict['model']]}," + ",".join(args)
                str_args = str_args.replace(' ','')
                f.write(str_args)
                f.write("\n")
                i+=1

### Hyperparams comp between runs

In [34]:
ldf = pd.read_csv(LOG_FILE, index_col=0)
ldf['run'] = [1 if ind < CALMIP_START_INDEX[2] else 2 for ind in ldf.index]
model= 'iTransformer'
for seq_len in [16,24,32,48,64,96]:
    hp_cols = ['run','model','features','enc_in','seq_len','pred_len','learning_rate','dropout','optimizer','wd','e_layers','d_model','n_heads','d_layers','d_ff']
    indices = ldf[hp_cols].drop_duplicates().query(f"model == '{model}'").query("features == 'MS'").query(f"seq_len == {seq_len}").index
    display(ldf.loc[indices,hp_cols+['rmse','fde']])
    

,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
168,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,256,4,1,256,1.443561,2.323885
169,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,3,256,4,1,256,1.450601,2.332160
170,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,4,256,8,1,256,1.442348,2.322347
171,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,512,4,1,512,1.448953,2.330210
172,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,3,512,4,1,512,1.455063,2.341624
173,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,4,512,8,1,512,1.450734,2.334224
929,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,256,4,1,256,0.642176,1.198726
932,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,3,256,4,1,256,0.641774,1.195672
935,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,4,256,8,1,256,0.682622,1.258449
939,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
143,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,256,4,1,256,1.995806,3.211073
144,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,256,4,1,256,2.001076,3.219440
146,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,4,256,8,1,256,1.992844,3.202685
148,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,512,4,1,512,1.996784,3.207222
151,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,512,4,1,512,2.003004,3.221637
154,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,4,512,8,1,512,1.997576,3.212950
963,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,256,4,1,256,1.512109,2.615828
969,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
973,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
976,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,512,4,1,512,NaN,NaN


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
7,1,iTransformer,MS,61,32,32,0.001,0.1,adamw,0.01,2,256,8,1,512,2.431306,3.858518
62,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,2,256,4,1,256,2.436089,3.866682
63,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,3,256,4,1,256,2.432088,3.861075
64,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,4,256,8,1,256,2.435086,3.862606
65,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,2,512,4,1,512,2.436152,3.864174
66,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,3,512,4,1,512,2.438879,3.864929
67,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,4,512,8,1,512,2.439884,3.872389
989,2,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,2,256,4,1,256,1.734917,3.077791
996,2,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
997,2,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,4,256,8,1,256,1.741516,3.077881


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
252,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,256,4,1,256,3.096723,4.794221
255,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,3,256,4,1,256,3.096600,4.793936
256,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,4,256,8,1,256,3.097650,4.797655
257,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
258,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,3,512,4,1,512,3.102762,4.805478
260,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,4,512,8,1,512,3.094412,4.788723
1025,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,256,4,1,256,NaN,NaN
1033,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,3,256,4,1,256,2.654699,4.384897
1034,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,4,256,8,1,256,2.711016,4.450923
1038,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
284,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,256,4,1,256,3.677281,5.712017
285,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,3,256,4,1,256,3.675936,5.708266
286,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,4,256,8,1,256,3.684022,5.720809
287,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
288,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,3,512,4,1,512,NaN,NaN
290,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,4,512,8,1,512,3.679391,5.716301
1064,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,256,4,1,256,NaN,NaN
1065,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
1072,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
1073,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,4,256,8,1,256,3.238571,5.058647


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
366,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,256,4,1,256,4.937085,7.675739
367,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,3,256,4,1,256,4.935859,7.676727
368,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,4,256,8,1,256,4.935022,7.671932
369,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
370,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,3,512,4,1,512,NaN,NaN
371,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,4,512,8,1,512,NaN,NaN
1096,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,256,4,1,256,NaN,NaN
1105,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
1106,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,4,256,8,1,256,4.349303,6.664582
1110,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
